In [ ]:
# default_exp feeder_ops
# default_cls_lvl 2

# Feeder Ops

> Functions for controlling the feeding machine

We will try to use different solutions for predicting pictures and deciding what we are going to do about them.
- use a FileWatcher
- 

In [ ]:
#export
import os
import argparse
from datetime import datetime, timedelta, date
import time
import logging
import numpy as np
import cv2
import sys
import paramiko
from pathlib import Path
from logging.handlers import RotatingFileHandler
from dotenv import load_dotenv
from fastai.vision import *
from fastscript import *
from astral import LocationInfo
from astral.sun import sun
from dateutil import tz

In [ ]:
#export
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())


CAP_URL = os.getenv('CAP_URL', default='.')
PATH_SAVE_FILES = os.getenv('PATH_SAVE_FILES', default='data')
PATH_FILE_LOG = os.getenv('PATH_FILE_LOG', default='data/camera_log')
SLEEP_TIME_BETWEEN_CAPTURE = float(os.getenv('SLEEP_TIME_BETWEEN_CAPTURE', default=0.25))


ACTIVATE_PREDICTION = bool(os.getenv('ACTIVATE_PREDICTION', default=False))
SLEEP_BETWEEN_FEEDING = int(os.getenv('SLEEP_BETWEEN_FEEDING', default=3600))
PREDICTION_THRES = float(os.getenv('PREDICTION_THRES', default=0.98))
MAX_FEEDING_SESSIONS_PER_DAY = int(os.getenv('MAX_FEEDING_SESSIONS_PER_DAY', default=4))
ACTIVATE_FEEDER = bool(os.getenv('ACTIVATE_FEEDER', default=False))
PATH_MODEL = os.getenv('PATH_MODEL', default='data')
PATH_FOLDER_SUCCESSFUL_PREDICTIONS = os.getenv('PATH_FOLDER_SUCCESSFUL_PREDICTIONS', default='data/successful')

FEEDER_URL = os.getenv('FEEDER_URL')
FEEDER_USER = os.getenv('FEEDER_USER')
FEEDER_PWD = os.getenv('FEEDER_PWD')
FEEDER_CMD = os.getenv('FEEDER_CMD', default='echo test')



In [ ]:
FEEDER_CMD

'sudo python3 test2.py'

In [ ]:
#export
def get_logger(log_file=None):
    """
    Initialize global logger and return it.

    :param log_file: log to this file, or to standard output if None
    :return: created logger
    """
    
    formatter = logging.Formatter(
        fmt='%(asctime)s %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
    
    log = logging.getLogger()
    if len(log.handlers) >= 2:
        return log
        
    log.setLevel(logging.INFO)
    if log_file is not None:
        os.makedirs(os.path.dirname(log_file), exist_ok=True)
        handler = RotatingFileHandler(
            log_file,
            maxBytes=1024*1024*30,
            backupCount=3)
        handler.setFormatter(formatter)
        log.addHandler(handler)
        handler.setLevel(logging.DEBUG)

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.setLevel(logging.INFO)
    log.addHandler(handler)

    return log

In [ ]:
#export
def create_folder(path: Path, foldername):
    """ creates a folder if it does not exist already """
    (path / foldername).mkdir(parents=True, exist_ok=True)
    return path / foldername

In [ ]:
model = load_learner(PATH_MODEL)
model.predict(open_image('data/horse_poo/20181215091846-poo_day.jpg'))[2][0].numpy() > 0

True

In [ ]:
time_start = int(s["sunrise"].astimezone(tz.tzlocal()).strftime('%H%M'))

time_stop = int(s["sunset"].astimezone(tz.tzlocal()).strftime('%H%M'))
time_stop
    

1822

In [ ]:
#export
def get_sunrise():
    """gets the sunrise as an int"""
    city = LocationInfo("Ludesch", "Austria", "Europe/Berlin", 47.2, 9.7)
    s = sun(city.observer, date= datetime.now())
    return int(s["sunrise"].astimezone(tz.tzlocal()).strftime('%H%M'))


def get_sunset():
    """gets the sunset as an int"""
    city = LocationInfo("Ludesch", "Austria", "Europe/Berlin", 47.2, 9.7)
    s = sun(city.observer, date= datetime.now())
    return int(s["sunset"].astimezone(tz.tzlocal()).strftime('%H%M'))

In [ ]:
get_sunrise()

639

In [ ]:
#export
@call_parse
def cap_and_predict():
    """this function captures camera pictures. To configure this function please use a .env file"""
    
    
        
    logger = get_logger(PATH_FILE_LOG)

    path = Path(PATH_SAVE_FILES)    
    
    current_date = datetime.today().date()
    prev_date = datetime.min.date
    
    
    last_run_feeder = datetime.min
    num_run_feeder_per_day = 0
    
    model = None
    if ACTIVATE_PREDICTION:
        model = load_learner(PATH_MODEL)
    
    while True:
        time.sleep(10)
        logger.info('outer while loop')

        try:
            cap = cv2.VideoCapture(CAP_URL)
            cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
            while cap.isOpened():
                
                ret, pic = cap.read()
                
                if ret is False:
                    logger.warning('pic not readable')
                    break

                now = datetime.now()
                current_date = datetime.today().date()                
                time_current = int(now.strftime('%H%M'))
                
                if get_sunrise() > time_current:
                    continue

                if get_sunset() < time_current:
                    continue

                if prev_date != current_date:
                    num_run_feeder_per_day = 0                    
                    fld_save_to = create_folder(path, current_date.strftime('%Y%m%d'))
                    logger.info(f"msg=update folder to {current_date}")
                
                now_str = now.strftime('%Y%m%d%H%M%S_%f')
                path_save_file = fld_save_to / (now_str + '.jpg')
                cv2.imwrite(str(path_save_file), pic)
                prev_date = datetime.today().date() #assign new date to compare to
                
                prediction = None
                if ACTIVATE_PREDICTION:
                    img = open_image(path_save_file)
                    prediction = model.predict(img)[2][0].numpy()
                    folder =  Path(PATH_FOLDER_SUCCESSFUL_PREDICTIONS) / current_date.strftime('%Y%m%d')
                    if prediction >= PREDICTION_THRES:
                        folder = create_folder(Path(PATH_FOLDER_SUCCESSFUL_PREDICTIONS), 
                                               current_date.strftime('%Y%m%d'))
                        img.save(folder / path_save_file.name)
                
                if ACTIVATE_FEEDER:
                    if PREDICTION_THRES <= prediction:
                        if (last_run_feeder + timedelta(seconds=SLEEP_BETWEEN_FEEDING) < datetime.now() 
                            and num_run_feeder_per_day <= MAX_FEEDING_SESSIONS_PER_DAY):
                            log.info("running feeder")
                            last_run_feeder = datetime.now()
                            num_run_feeder_per_day += 1
                            run_feeder()                       
                   
                
                time.sleep(SLEEP_TIME_BETWEEN_CAPTURE)
                
        except Exception as e:
            logger.error(e)

        finally:
            cap.release()
            cv2.destroyAllWindows()

In [ ]:
last_run_feeder = datetime.min
last_run_feeder = datetime.now()
last_run_feeder += timedelta(seconds=SLEEP_BETWEEN_FEEDING)
last_run_feeder



datetime.datetime(2020, 3, 12, 10, 28, 15, 201775)

In [ ]:
#export
def run_feeder():
    """runs feeder"""
    log = get_logger(PATH_FILE_LOG)
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(FEEDER_URL, username=FEEDER_USER, password=FEEDER_PWD)
    ssh_stdin, ssh_stdout, ssh_stder = ssh.exec_command(FEEDER_CMD)
    log.info(ssh_stdout.read())
    log.info(ssh_stder.read())


In [ ]:
run_feeder()

2020-03-12 09:28:17 Connected (version 2.0, client OpenSSH_7.4p1)
2020-03-12 09:28:17 Authentication (publickey) successful!
2020-03-12 09:28:18 b'test running engine\n'
2020-03-12 09:28:18 b''


In [ ]:
current_date = datetime.today().date()
folder = create_folder(Path(PATH_FOLDER_SUCCESSFUL_PREDICTIONS), current_date.strftime('%Y%m%d'))

In [ ]:
from nbdev.export import *
notebook2script('04_Feeder_Ops.ipynb')


Converted 04_Feeder_Ops.ipynb.


In [ ]:
!python PooDetector/feeder_ops.py